In [1]:
## Importing Libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [2]:
train=pd.read_csv("Data/myntra_train_dataset.csv")
test=pd.read_csv("Data/myntra_test.csv")

In [5]:
X_train=np.load("Train_64_64.npy")

In [6]:
X_train.shape

(69897, 64, 64, 3)

In [7]:
X_test=np.load("Test_64_64.npy")

In [8]:
X_test.shape

(14952, 64, 64, 3)

In [15]:
mean_img=np.mean(X_train,axis=0)
std_img=np.std(X_train,axis=0)
X_train_norm=(X_train-mean_img)/std_img
X_test_norm=(X_test-mean_img)/std_img

In [12]:
train=pd.read_csv("Cleaned_Train_Data.csv")

In [13]:
train.head(2)

,Unnamed: 0,Brand,Category,Gender,Color,Image_Names,Sub_category
0,0_Graphic.jpg,Roadster,Tshirts,Men,Black,0_Graphic.jpg,Graphic
1,1_Graphic.jpg,Roadster,Tshirts,Men,Black,1_Graphic.jpg,Graphic


In [16]:
labels= train['Sub_category'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(labels))}
y_train = [Y_train[k] for k in labels]
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [9]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping

In [17]:
model = Sequential()
model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (64,64,3))) # if you resize the image above, change the shape
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [18]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
model.fit(X_train_norm, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])

Train on 48927 samples, validate on 20970 samples
Epoch 1/10
48927/48927 [==============================] - 500s 10ms/step - loss: 1.8733 - acc: 0.4745 - val_loss: 1.6680 - val_acc: 0.5349
Epoch 2/10
48927/48927 [==============================] - 298s 6ms/step - loss: 1.5035 - acc: 0.5810 - val_loss: 1.5556 - val_acc: 0.5573
Epoch 3/10
48927/48927 [==============================] - 300s 6ms/step - loss: 1.4050 - acc: 0.6037 - val_loss: 1.5052 - val_acc: 0.5722
Epoch 4/10
48927/48927 [==============================] - 300s 6ms/step - loss: 1.3389 - acc: 0.6191 - val_loss: 1.5121 - val_acc: 0.5750
Epoch 5/10
48927/48927 [==============================] - 300s 6ms/step - loss: 1.2781 - acc: 0.6337 - val_loss: 1.5090 - val_acc: 0.5655
Epoch 6/10
48927/48927 [==============================] - 313s 6ms/step - loss: 1.2238 - acc: 0.6478 - val_loss: 1.5309 - val_acc: 0.5751
Epoch 7/10
48927/48927 [==============================] - 298s 6ms/step - loss: 1.1720 - acc: 0.6599 - val_loss: 1.5346 -

In [19]:
predictions=model.predict(X_test_norm)

In [24]:
predictions=np.argmax(predictions,axis=1)

In [25]:
y_maps = dict()
y_maps = {v:k for k, v in Y_train.items()}
pred_labels = [y_maps[k] for k in predictions]

## Submission

In [26]:
test=pd.read_csv("Cleaned_Test_Data.csv")

In [27]:
test.head(2)

,Brand,Category,Gender,Color,Image_Names
0,Allen Solly,Tshirts,Men,Red,0.jpg
1,Celio,Tshirts,Men,Yellow,1.jpg


In [32]:
df_sub=pd.DataFrame({"Image_Names":test["Image_Names"],"Brand":test["Brand"],"Label":pred_labels})

In [33]:
sub=pd.read_csv("Data/Submission_online.csv")

In [34]:
sub.head()

,Brand,Category,Gender,Color,Link_to_the_image,Sub_category
0,Allen Solly,Tshirts,Men,Red,http://myntra.myntassets.com/assets/images/182...,NaN
1,Celio,Tshirts,Men,Yellow,http://myntra.myntassets.com/assets/images/109...,NaN
2,CULT FICTION,Tshirts,Men,Rust,http://myntra.myntassets.com/assets/images/159...,NaN
3,Antigravity,Tshirts,Women,Blue,http://myntra.myntassets.com/assets/images/185...,NaN
4,Being Human,Tshirts,Men,Charcoal,http://myntra.myntassets.com/assets/images/100...,NaN


In [41]:
test["Sub_category"]=pred_labels

In [52]:
sub_2=pd.merge(sub,test,on=["Brand","Category","Gender","Color",]).iloc[:15000,]

In [53]:
sub_2.columns

Index(['Brand', 'Category', 'Gender', 'Color', 'Link_to_the_image',
       'Sub_category_x', 'Image_Names', 'Sub_category_y'],
      dtype='object')

In [55]:
sub_2=sub_2[['Brand', 'Category', 'Gender', 'Color', 'Link_to_the_image','Sub_category_y']]

In [57]:
sub_2.rename(columns={"Sub_category_y":"Sub_category"},inplace=True)

In [58]:
sub_2.to_csv("Sub_64_64.csv",index=False)

In [59]:
df_sub.to_csv("Main_submission_file.csv",index=False)

In [60]:
from keras.models import model_from_json

In [61]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_64_64_10E.h5")
print("Saved model to disk")

Saved model to disk


In [62]:
sub=pd.read_csv("Submission Files/First_submission.csv")

In [66]:
sub.head(10)

,Brand,Category,Gender,Color,Link_to_the_image,Sub_category
0,Allen Solly,Tshirts,Men,Red,http://myntra.myntassets.com/assets/images/182...,Solid
1,Celio,Tshirts,Men,Yellow,http://myntra.myntassets.com/assets/images/109...,Solid
2,CULT FICTION,Tshirts,Men,Rust,http://myntra.myntassets.com/assets/images/159...,Solid
3,Antigravity,Tshirts,Women,Blue,http://myntra.myntassets.com/assets/images/185...,Typography
4,Being Human,Tshirts,Men,Charcoal,http://myntra.myntassets.com/assets/images/100...,Typography
5,Adamo London,Tshirts,Men,Red,http://myntra.myntassets.com/assets/images/161...,Typography
6,Aeropostale,Tshirts,Women,White,http://myntra.myntassets.com/assets/images/206...,Typography
7,ESPRIT,Tshirts,Men,Navy Blue,http://myntra.myntassets.com/assets/images/234...,Solid
8,2GO,Tshirts,Women,Grey,http://myntra.myntassets.com/assets/images/156...,Solid
9,American Crew,Tshirts,Men,Grey Melange,http://myntra.myntassets.com/assets/images/128...,Solid


In [65]:
test.head(10)

,Brand,Category,Gender,Color,Image_Names,Sub_category
0,Allen Solly,Tshirts,Men,Red,0.jpg,Solid
1,Celio,Tshirts,Men,Yellow,1.jpg,Solid
2,CULT FICTION,Tshirts,Men,Rust,2.jpg,Solid
3,Antigravity,Tshirts,Women,Blue,3.jpg,Graphic
4,Being Human,Tshirts,Men,Charcoal,4.jpg,Graphic
5,Adamo London,Tshirts,Men,Red,5.jpg,Abstract
6,Aeropostale,Tshirts,Women,White,6.jpg,Typography
7,ESPRIT,Tshirts,Men,Navy Blue,7.jpg,Conversational
8,2GO,Tshirts,Women,Grey,8.jpg,Solid
9,American Crew,Tshirts,Men,Grey Melange,9.jpg,Solid


## 128 X 128 Image Training

In [2]:
import numpy as np
import pandas as pd
import sklearn
import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train=pd.read_csv("Cleaned_Train_Data.csv")
test=pd.read_csv("Cleaned_Test_Data.csv")

In [5]:
train.head()

,Unnamed: 0,Brand,Category,Gender,Color,Image_Names,Sub_category
0,0_Graphic.jpg,Roadster,Tshirts,Men,Black,0_Graphic.jpg,Graphic
1,1_Graphic.jpg,Roadster,Tshirts,Men,Black,1_Graphic.jpg,Graphic
2,2_Biker.jpg,Roadster,Tshirts,Men,Black,2_Biker.jpg,Biker
3,3_Graphic.jpg,Roadster,Tshirts,Men,Black,3_Graphic.jpg,Graphic
4,4_Graphic.jpg,Roadster,Tshirts,Men,Black,4_Graphic.jpg,Graphic


In [6]:
X_train=np.load("Numpy_Saved_Arrays/Train_128_128.npy")

In [7]:
X_test=np.load("Numpy_Saved_Arrays/Test_128_128.npy")

In [8]:
X_test.shape

(14952, 128, 128, 3)

In [9]:
X_train.shape

(69897, 128, 128, 3)

In [10]:
train.head(2)

,Unnamed: 0,Brand,Category,Gender,Color,Image_Names,Sub_category
0,0_Graphic.jpg,Roadster,Tshirts,Men,Black,0_Graphic.jpg,Graphic
1,1_Graphic.jpg,Roadster,Tshirts,Men,Black,1_Graphic.jpg,Graphic


In [13]:
set(train['Sub_category'].values)

{'Abstract',
 'Biker',
 'Camouflage',
 'Checked',
 'Colourblocked',
 'Conversational',
 'Floral',
 'Geometric',
 'Graphic',
 'Horizontal Stripes',
 'Humour and Comic',
 'Music',
 'People and Places',
 'Polka Dots',
 'Self Design',
 'Solid',
 'Sports',
 'Sports and Team Jersey',
 'Striped',
 'Superhero',
 'Tie and Dye',
 'Tribal',
 'Typography',
 'Varsity'}

In [16]:
class_list=train['Sub_category'].tolist()

In [18]:
Y_train={k:v+1 for v,k in enumerate(set(class_list))}

In [19]:
y_train = [Y_train[k] for k in class_list]

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

In [21]:
y_train=to_categorical(y_train)

In [22]:
model = Sequential()
model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (128,128,3))) # if you resize the image above, change the shape
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [23]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')

In [ ]:
model.fit(X_train, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])

Train on 48927 samples, validate on 20970 samples
Epoch 1/10
48927/48927 [==============================] - 2900s 59ms/step - loss: 1.7769 - acc: 0.5079 - val_loss: 1.5771 - val_acc: 0.5526
Epoch 2/10
48927/48927 [==============================] - 2141s 44ms/step - loss: 1.4485 - acc: 0.5956 - val_loss: 1.5416 - val_acc: 0.5588
Epoch 3/10
48927/48927 [==============================] - 2478s 51ms/step - loss: 1.3426 - acc: 0.6158 - val_loss: 1.5076 - val_acc: 0.5661
Epoch 4/10
48927/48927 [==============================] - 3020s 62ms/step - loss: 1.2628 - acc: 0.6382 - val_loss: 1.4647 - val_acc: 0.5745
Epoch 5/10
48927/48927 [==============================] - 4881s 100ms/step - loss: 1.1952 - acc: 0.6541 - val_loss: 1.4854 - val_acc: 0.5800
Epoch 6/10
 2000/48927 [>.............................] - ETA: 1:23:14 - loss: 1.0748 - acc: 0.6780

## 30 th March

In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [ ]:
np.save